In [1]:
import pandas as pd
import numpy as np
import os
import io
import gzip

In [2]:
# Cargamos los conjuntos de datos disponibles en el enunciado y vemos los datos
df = pd.read_csv('../../../data/raw/train.csv')
df_test_nolabel = pd.read_csv('../../../data/raw/test_nolabel.csv')
df_sample_submission= pd.read_csv('../../../data/raw/sample_submission.csv')
df

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,1-Nov-05,2006,2,...,0,2,0,1,N,N,31-Dec-05,"$8,000.00",$0.00,1
1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,6-Jun-05,2005,2,...,1,2,1,1,N,N,31-Jul-05,"$166,000.00",$0.00,1
2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,18-Mar-03,2003,2,...,4,2,1,2,Y,N,31-Mar-03,"$25,000.00",$0.00,1
3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,28-Jun-95,1995,2,...,0,0,1,0,N,N,31-Jan-96,"$220,100.00",$0.00,1
4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,16-Apr-09,2009,0,...,0,0,0,1,N,N,31-May-09,"$25,000.00",$0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,4f9443d2a46,1573725008,"SIBILA RACE ENGINEERING, INC",MASSILLON,OH,CITIZENS BANK NATL ASSOC,RI,9-Dec-05,2006,1,...,0,1,0,1,N,N,31-Dec-05,"$70,000.00",$0.00,1
22831,798db2753a7,2011184008,ENVIRO SHIELD POWER WASHING,SPRINGBORO,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,27-Apr-98,1998,2,...,0,0,1,0,N,Y,31-May-98,"$30,000.00",$0.00,1
22832,ddb3c5e9bff,4082983001,"MAINLINE TRCK&TRAILR SRVC, INC",BEDFORD,OH,GROWTH CAPITAL CORP.,OH,9-May-90,1990,16,...,6,10,1,0,N,N,13-Feb-91,"$92,000.00",$0.00,1
22833,407200a5dfe,7783283010,TIN BOX STUDIO,CINCINNATI,OH,KEYBANK NATIONAL ASSOCIATION,OH,10-Nov-94,1995,1,...,0,0,1,0,N,Y,31-Jan-95,"$20,000.00",$0.00,1


In [3]:
# Comprobamos que los datos son únicos y eliminamos las columnas no relevantes
df_train = df.copy()
df_train.drop(columns=['State'], inplace=True)
# df_train.columns.tolist()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22835 entries, 0 to 22834
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 22835 non-null  object 
 1   LoanNr_ChkDgt      22835 non-null  int64  
 2   Name               22834 non-null  object 
 3   City               22834 non-null  object 
 4   Bank               22813 non-null  object 
 5   BankState          22813 non-null  object 
 6   ApprovalDate       22835 non-null  object 
 7   ApprovalFY         22835 non-null  int64  
 8   NoEmp              22835 non-null  int64  
 9   NewExist           22833 non-null  float64
 10  CreateJob          22835 non-null  int64  
 11  RetainedJob        22835 non-null  int64  
 12  FranchiseCode      22835 non-null  int64  
 13  UrbanRural         22835 non-null  int64  
 14  RevLineCr          22744 non-null  object 
 15  LowDoc             22792 non-null  object 
 16  DisbursementDate   227

Para cada variable, explorar: 
- Tipo de datos
- Valores nulos
- Valores únicos
- Outliers

## Ciudad - "City"

In [4]:
# Ciudad - Valores unicos
df["City"].nunique() # 1678

1678

In [5]:
df["City"].isnull().sum() # Un valor nulo - eliminar
# Eliminar filas con valores nulos
df_train = df_train.dropna(subset=["City"])

In [6]:
# Convertir la columna 'City' a mayúsculas y ver duplicados
df_train["City"] = df_train["City"].str.upper()
df_train["City"].nunique() 

1207

In [7]:
# Filas con números
df_train[df_train["City"].str.contains(r"\d", regex=True, na=False)] # 3 filas - eliminar
df_train = df_train[~df_train["City"].str.contains(r"\d", regex=True, na=False)]

In [8]:
df_train["City"].nunique() # 1204

1204

In [9]:
# Detectar valores que contienen texto entre paréntesis (permitiendo espacios)
ciudades_con_texto_parentesis = df_train[df_train["City"].str.contains(r" \(.*\)", regex=True, na=False)]
# print(ciudades_con_texto_parentesis["City"])

ciudades_con_texto_parentesis = ciudades_con_texto_parentesis["City"].str.replace(r" \(.*", "", regex=True).str.strip()
# print(ciudades_con_texto_parentesis.unique())
df_train["City"] = df_train["City"].str.replace(r" \(.*", "", regex=True).str.strip()
df_train["City"].nunique() #1181

1181

In [10]:
# Filtrar filas donde 'City' contiene una coma
ciudades_con_comas = df_train[df_train["City"].str.contains(",", regex=False, na=False)]
# Eliminar las filas con comas al final
df_train["City"] = df_train["City"].str.replace(r",$", "", regex=True).str.strip()
# Filtrar filas donde 'City' contiene una coma
ciudades_con_comas = df_train[df_train["City"].str.contains(",", regex=False, na=False)]
ciudades_con_comas # queda uno
df_train["City"].nunique() #1174

1174

### Conclusión:

La columna "City" contenía valores con diversos formatos y caracteres especiales, que dificultaban la agrupación. Se ha disminuido el número de categorías en la característica "City" para facilitar el entrenamiento y determinar si es relevante agrupar por ciudad. 

In [11]:
# AYUDAR PARA AGRUPAR POR CIUDADES (p.e. top 20)

df_train.shape[0] # 22835 filas iniciales - se han formateado datos, no eliminado.# Obtener frecuencias de ciudades
frecuencia_ciudades = df_train["City"].value_counts()

# Calcular percentiles específicos
percentiles = frecuencia_ciudades.quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

# Mostrar los percentiles
print(percentiles)

0.25      1.00
0.50      3.00
0.75     10.00
0.90     40.00
0.95     79.00
0.99    166.16
Name: count, dtype: float64


## Bank

In [12]:
# Bank - Valores unicos
df["Bank"].nunique() # 286

286

In [13]:
df_train["Bank"] = df_train["Bank"].str.upper() # no hay texto en minúsculas
df_train["Bank"].nunique() 

286

In [14]:
df["Bank"].isnull().sum() # 22 - eliminar
# Eliminar filas con valores nulos
df_train = df_train.dropna(subset=["Bank"])
df_train.shape[0]

22809

In [15]:
df_train["Bank"].value_counts()

Bank
THE HUNTINGTON NATIONAL BANK      4261
PNC BANK, NATIONAL ASSOCIATION    2901
JPMORGAN CHASE BANK NATL ASSOC    2706
CITIZENS BANK NATL ASSOC          1734
U.S. BANK NATIONAL ASSOCIATION    1416
                                  ... 
PORTAGEBANC SHARES, INC              1
UNITY BANK                           1
THE KILLBUCK SAVINGS BANK CO.        1
THE HOCKING VALLEY BANK              1
CITIZENS BANK OF PENNSYLVANIA        1
Name: count, Length: 286, dtype: int64

### Conclusión Bank
No parece muy relevante. Variable categórica con 286 valores, posible agrupación p.e. top 10 de bancos o nueva variable "tamaño del banco"

## BankState

In [16]:
# Bank - Valores unicos
df_train["BankState"].nunique()

41

In [17]:
df_train["BankState"].isnull().sum()

0

In [18]:
# Detectar valores en 'BankState' que tienen más de dos letras (no son abreviaturas de estado válidas)
bankstate_largo = df_train[df_train["BankState"].str.len() > 2]["BankState"].unique()

In [21]:
# Obtener los 10 estados más frecuentes
frec_bankstates= df_train["BankState"].value_counts()
frec_bankstates

BankState
OH    14872
IL     2659
RI     1739
DE     1328
VA      502
CA      358
IN      246
SD      228
PA      126
TX      106
WV       68
SC       65
OR       64
KY       57
FL       55
CT       52
MI       46
NY       46
NC       44
MN       37
KS       29
TN       20
UT       11
NV       10
DC        7
WA        6
MO        6
CO        4
NJ        3
OK        2
GA        2
ID        2
ME        1
AZ        1
IA        1
MD        1
NE        1
AL        1
WI        1
WY        1
NM        1
Name: count, dtype: int64

### Conclusión

Variable candidata a hacer agrupaciones que puedan ser relevantes. Categorías "reducidas" bien formateada, prácticamente sin elementos nulos. Ver la distribución de Accept en funcion de BankState